Credits: this notebook belongs to [Practical DL](https://docs.google.com/forms/d/e/1FAIpQLScvrVtuwrHSlxWqHnLt1V-_7h2eON_mlRR6MUb3xEe5x9LuoA/viewform?usp=sf_link) course by Yandex School of Data Analysis.

In [1]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [2]:
class Module(object):
    """
    Basically, you can think of a module as of a something (black box)
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`:

        output = module.forward(input)

    The module should be able to perform a backward pass: to differentiate the `forward` function.
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule.

        gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True

    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.

        This includes
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput


    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.

        Make sure to both store the data in `output` field and return it.
        """

        # The easiest case:

        # self.output = input
        # return self.output

        pass

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input.
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.

        The shape of `gradInput` is always the same as the shape of `input`.

        Make sure to both store the gradients in `gradInput` field and return it.
        """

        # The easiest case:

        # self.gradInput = gradOutput
        # return self.gradInput

        pass

    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass

    def zeroGradParameters(self):
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass

    def getParameters(self):
        """
        Returns a list with its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def train(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True

    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [4]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially.

         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`.
    """

    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []

    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:

            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})


        Just write a little loop.
        """

        # Your code goes here. ################################################
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:

            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)
            gradInput = module[0].backward(input, g_1)


        !!!

        To ech module you need to provide the input, module saw while forward pass,
        it is used while computing gradients.
        Make sure that the input for `i-th` layer the output of `module[i]` (just the same input as in forward pass)
        and NOT `input` to this Sequential module.

        !!!

        """
        # Your code goes here. ################################################
        def updateOutput(self, input):
              self.output = input
              for module in self.modules:
                self.output = module.forward(self.output)
                return self.output
                return self.gradInput

        def backward(self, input, gradOutput):
          current_grad = gradOutput
          for i in range(len(self.modules)-1, -1, -1):
            current_grad = self.modules[i].backward(
                self.modules[i-1].output if i > 0 else input,
                current_grad
                )
            self.gradInput = current_grad
            return self.gradInput


    def zeroGradParameters(self):
        for module in self.modules:
            module.zeroGradParameters()

    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]

    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]

    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string

    def __getitem__(self,x):
        return self.modules.__getitem__(x)

    def train(self):
        """
        Propagates training parameter through all modules
        """
        self.training = True
        for module in self.modules:
            module.train()

    def evaluate(self):
        """
        Propagates training parameter through all modules
        """
        self.training = False
        for module in self.modules:
            module.evaluate()

# Layers

## 1. Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [5]:
class Linear(Module):
    """
    A module which applies a linear transformation
    A common name is fully-connected layer, InnerProductLayer in caffe.

    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()

        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)

        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.dot(input, self.W.T) + self.b
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.dot(gradOutput, self.W)
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradW += np.dot(gradOutput.T, input)
        self.gradb += np.sum(gradOutput, axis=0)
        pass

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

## 2. SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

In [8]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()

    def updateOutput(self, input):
        # start with normalization for numerical stability
        exp_input = np.exp(input - np.max(input, axis=1, keepdims=True))
        self.output = exp_input / np.sum(exp_input, axis=1, keepdims=True)

        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
            batch_size = input.shape[0]
            s = np.sum(gradOutput * self.output, axis=1, keepdims=True)
            self.gradInput = self.output * (gradOutput - s)

            return self.gradInput

    def __repr__(self):
        return "SoftMax"

## 3. LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

In [9]:
class LogSoftMax(Module):
    def __init__(self):
         super(LogSoftMax, self).__init__()

    def updateOutput(self, input):
        # start with normalization for numerical stability
        shifted = input - np.max(input, axis=1, keepdims=True)
        exp_shifted = np.exp(shifted)
        log_sum = np.log(np.sum(exp_shifted, axis=1, keepdims=True))
        self.output = shifted - log_sum
        return self.output


    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        exp_shifted = np.exp(self.output)  # softmax
        s = np.sum(gradOutput, axis=1, keepdims=True)
        self.gradInput = gradOutput - exp_shifted * s
        return self.gradInput

    def __repr__(self):
        return "LogSoftMax"

## 4. Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance:
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance.

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

In [12]:
class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None
        self.moving_variance = None

    def updateOutput(self, input):
        if self.training:
            batch_mean = np.mean(input, axis=0)
            batch_var = np.mean((input - batch_mean) ** 2, axis=0)

            if self.moving_mean is None:
                self.moving_mean = batch_mean.copy()
                self.moving_variance = batch_var.copy()
            else:
                # Обновляем скользящие средние
                self.moving_mean = self.alpha * self.moving_mean + (1 - self.alpha) * batch_mean
                self.moving_variance = self.alpha * self.moving_variance + (1 - self.alpha) * batch_var

            # Нормализуем
            self.output = (input - batch_mean) / np.sqrt(batch_var + self.EPS)

            # Сохраняем для backward
            self.batch_mean = batch_mean
            self.batch_var = batch_var
            self.batch_std = np.sqrt(batch_var + self.EPS)

        else:
            # В режиме оценки используем скользящие статистики
            if self.moving_mean is None:
                # Если не было обучения, просто пропускаем
                self.output = input
            else:
                self.output = (input - self.moving_mean) / np.sqrt(self.moving_variance + self.EPS)

        return self.output

    def updateGradInput(self, input, gradOutput):
        if self.training:
            N = input.shape[0]

            dxhat = gradOutput / self.batch_std

            dvar = np.sum(gradOutput * (input - self.batch_mean) * -0.5 * (self.batch_var + self.EPS)**(-1.5), axis=0)

            dmean = np.sum(gradOutput * -1 / self.batch_std, axis=0) + dvar * np.mean(-2 * (input - self.batch_mean), axis=0)

            self.gradInput = dxhat + dvar * 2 * (input - self.batch_mean) / N + dmean / N

        else:
            # В режиме оценки градиент просто проходит через нормализацию
            if self.moving_mean is None:
                self.gradInput = gradOutput
            else:
                self.gradInput = gradOutput / np.sqrt(self.moving_variance + self.EPS)

        return self.gradInput

    def __repr__(self):
        return "BatchNormalization"

In [13]:
class ChannelwiseScaling(Module):
    """
       Implements linear transform of input y = gamma * x + beta
       where gamma, beta - learnable vectors of length x.shape[-1]
    """
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)

        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def updateOutput(self, input):
        self.output = input * self.gamma + self.beta
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.gamma
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        self.gradBeta = np.sum(gradOutput, axis=0)
        self.gradGamma = np.sum(gradOutput * input, axis=0)

    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)

    def getParameters(self):
        return [self.gamma, self.beta]

    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        return "ChannelwiseScaling"

Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 5. Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [14]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()

        self.p = p
        self.mask = None

    def updateOutput(self, input):
      if self.training:
        # Создаем маску и масштабируем
        self.mask = (np.random.rand(*input.shape) > self.p) / (1 - self.p)
        self.output = input * self.mask
      else:
        self.output = input
      return self.output

    def updateGradInput(self, input, gradOutput):
      if self.training:
        self.gradInput = gradOutput * self.mask
      else:
        self.gradInput = gradOutput
      return self.gradInput

    def __repr__(self):
        return "Dropout"

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**):

In [15]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()

    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput

    def __repr__(self):
        return "ReLU"

## 6. Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

In [18]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()

        self.slope = slope

    def updateOutput(self, input):
        self.output = np.where(input > 0, input, self.slope * input)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * np.where(input > 0, 1, self.slope)
        return self.gradInput

    def __repr__(self):
        return "LeakyReLU"

## 7. ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

In [19]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        super(ELU, self).__init__()

        self.alpha = alpha

    def updateOutput(self, input):
        self.output = np.where(input > 0, input, self.alpha * (np.exp(input) - 1))
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * np.where(input > 0, 1, self.alpha * np.exp(input))
        return self.gradInput

    def __repr__(self):
        return "ELU"

## 8. SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

In [20]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()

    def updateOutput(self, input):
        self.output = np.log(1 + np.exp(input))
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * (1 / (1 + np.exp(-input)))
        return self.gradInput

    def __repr__(self):
        return "SoftPlus"

# Criterions

Criterions are used to score the models answers.

In [21]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None

    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)

    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [22]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()

    def updateOutput(self, input, target):
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 9. Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula,
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



In [23]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def __init__(self):
        a = super(ClassNLLCriterionUnstable, self)
        super(ClassNLLCriterionUnstable, self).__init__()

    def updateOutput(self, input, target):
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        # Вычисляем отрицательный log likelihood
        loss = -np.sum(target * np.log(input_clamp)) / input.shape[0]
        self.output = loss
        return self.output

    def updateGradInput(self, input, target):
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        self.gradInput = -target / input_clamp / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterionUnstable"

## 10. Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

In [24]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()

    def updateOutput(self, input, target):
        # input уже log probabilities
        loss = -np.sum(target * input) / input.shape[0]
        self.output = loss
        return self.output

    def updateGradInput(self, input, target):
        self.gradInput = -target / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterion"

# Optimizers

### SGD optimizer with momentum
- `variables` - list of lists of variables (one list per layer)
- `gradients` - list of lists of current gradients (same structure as for `variables`, one array for each var)
- `config` - dict with optimization parameters (`learning_rate` and `momentum`)
- `state` - dict with optimizator state (used to save accumulated gradients)

In [25]:
def sgd_momentum(variables, gradients, config, state):
    # 'variables' and 'gradients' have complex structure, accumulated_grads will be stored in a simpler one
    state.setdefault('accumulated_grads', {})

    var_index = 0
    for current_layer_vars, current_layer_grads in zip(variables, gradients):
        for current_var, current_grad in zip(current_layer_vars, current_layer_grads):

            old_grad = state['accumulated_grads'].setdefault(var_index, np.zeros_like(current_grad))

            np.add(config['momentum'] * old_grad, config['learning_rate'] * current_grad, out=old_grad)

            current_var -= old_grad
            var_index += 1

## 11. [Adam](https://arxiv.org/pdf/1412.6980.pdf) optimizer
- `variables` - list of lists of variables (one list per layer)
- `gradients` - list of lists of current gradients (same structure as for `variables`, one array for each var)
- `config` - dict with optimization parameters (`learning_rate`, `beta1`, `beta2`, `epsilon`)
- `state` - dict with optimizator state (used to save 1st and 2nd moment for vars)

Formulas for optimizer:

Current step learning rate: $$\text{lr}_t = \text{learning_rate} * \frac{\sqrt{1-\beta_2^t}} {1-\beta_1^t}$$
First moment of var: $$\mu_t = \beta_1 * \mu_{t-1} + (1 - \beta_1)*g$$
Second moment of var: $$v_t = \beta_2 * v_{t-1} + (1 - \beta_2)*g*g$$
New values of var: $$\text{variable} = \text{variable} - \text{lr}_t * \frac{m_t}{\sqrt{v_t} + \epsilon}$$

In [26]:
def adam_optimizer(variables, gradients, config, state):
    state.setdefault('m', {})
    state.setdefault('v', {})
    state.setdefault('t', 0)
    state['t'] += 1

    var_index = 0
    lr_t = config['learning_rate'] * np.sqrt(1 - config['beta2']**state['t']) / (1 - config['beta1']**state['t'])

    for current_layer_vars, current_layer_grads in zip(variables, gradients):
        for current_var, current_grad in zip(current_layer_vars, current_layer_grads):
            var_first_moment = state['m'].setdefault(var_index, np.zeros_like(current_grad))
            var_second_moment = state['v'].setdefault(var_index, np.zeros_like(current_grad))

            # Обновляем моменты
            np.add(config['beta1'] * var_first_moment, (1 - config['beta1']) * current_grad, out=var_first_moment)
            np.add(config['beta2'] * var_second_moment, (1 - config['beta2']) * current_grad * current_grad, out=var_second_moment)

            # Обновляем параметры
            current_var -= lr_t * var_first_moment / (np.sqrt(var_second_moment) + config['epsilon'])

            var_index += 1


# Layers for advanced track homework
You **don't need** to implement it if you are working on `homework_main-basic.ipynb`

## 12. Conv2d [Advanced]
- input:   **`batch_size x in_channels x h x w`**
- output: **`batch_size x out_channels x h x w`**

You should implement something like pytorch `Conv2d` layer with `stride=1` and zero-padding outside of image using `scipy.signal.correlate` function.

Practical notes:
- While the layer name is "convolution", the most of neural network frameworks (including tensorflow and pytorch) implement operation that is called [correlation](https://en.wikipedia.org/wiki/Cross-correlation#Cross-correlation_of_deterministic_signals) in signal processing theory. So **don't use** `scipy.signal.convolve` since it implements [convolution](https://en.wikipedia.org/wiki/Convolution#Discrete_convolution) in terms of signal processing.
- It may be convenient to use `skimage.util.pad` for zero-padding.
- It's rather ok to implement convolution over 4d array using 2 nested loops: one over batch size dimension and another one over output filters dimension
- Having troubles with understanding how to implement the layer?
 - Check the last year video of lecture 3 (starting from ~1:14:20)
 - May the google be with you

In [27]:
import scipy as sp
import scipy.signal
import skimage.util

class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(Conv2d, self).__init__()
        # Проверяем, что размер ядра нечетный (для симметричного padding)
        assert kernel_size % 2 == 1, kernel_size

        # Инициализация весов по Xavier/He
        stdv = 1./np.sqrt(in_channels)
        # Веса: (выходные_каналы, входные_каналы, высота_ядра, ширина_ядра)
        self.W = np.random.uniform(-stdv, stdv, size = (out_channels, in_channels, kernel_size, kernel_size))
        # Смещения: по одному на каждый выходной канал
        self.b = np.random.uniform(-stdv, stdv, size=(out_channels,))
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        # Градиенты для весов и смещений
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def updateOutput(self, input):
        # Размер padding: половина от размера ядра (округление в меньшую сторону)
        pad_size = self.kernel_size // 2

        # 1. Добавляем zero-padding к входному тензору
        padded_input = np.pad(
            input,
            ((0, 0), (0, 0), (pad_size, pad_size), (pad_size, pad_size)),
            mode='constant',
            constant_values=0
        )

        # Получаем размеры входного тензора
        batch_size, in_channels, height, width = input.shape
        # Выходные размеры
        output_height = height
        output_width = width

        # Инициализируем выходной тензор
        self.output = np.zeros((batch_size, self.out_channels, output_height, output_width))

        # 2. Вычисляем свертку
        for b in range(batch_size):
            for oc in range(self.out_channels):
                for ic in range(self.in_channels):

                    self.output[b, oc] += sp.signal.correlate2d(
                        padded_input[b, ic],
                        self.W[oc, ic],
                        mode='valid'
                    )
                # 3. Добавляем смещение для каждого выходного канала
                self.output[b, oc] += self.b[oc]

        return self.output

    def updateGradInput(self, input, gradOutput):
        pad_size = self.kernel_size // 2
        batch_size, in_channels, height, width = input.shape

        # 1. Добавляем zero-padding к градиенту выхода для корректного вычисления градиента входа
        padded_grad = np.pad(
            gradOutput,
            ((0, 0), (0, 0), (pad_size, pad_size), (pad_size, pad_size)),
            mode='constant',
            constant_values=0
        )

        self.gradInput = np.zeros_like(input)

        # 2. Вычисляем градиент по входу
        for b in range(batch_size):
            for ic in range(self.in_channels):
                for oc in range(self.out_channels):
                    rotated_kernel = np.rot90(self.W[oc, ic], 2)

                    self.gradInput[b, ic] += sp.signal.correlate2d(
                        padded_grad[b, oc],
                        rotated_kernel,
                        mode='valid'
                    )

        return self.gradInput

    def accGradParameters(self, input, gradOutput):

        pad_size = self.kernel_size // 2

        # 1. Добавляем zero-padding ко входу для вычисления градиента весов
        padded_input = np.pad(
            input,
            ((0, 0), (0, 0), (pad_size, pad_size), (pad_size, pad_size)),
            mode='constant',
            constant_values=0
        )

        batch_size = input.shape[0]

        # 2. Вычисляем градиент по весам (dL/dW)
        for oc in range(self.out_channels):
            for ic in range(self.in_channels):
                for b in range(batch_size):
                    self.gradW[oc, ic] += sp.signal.correlate2d(
                        padded_input[b, ic],
                        gradOutput[b, oc],
                        mode='valid'
                    )

        # 3. Вычисляем градиент по смещениям (dL/db)

        self.gradb = np.sum(gradOutput, axis=(0, 2, 3))

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape

        q = 'Conv2d %d -> %d' %(s[1],s[0])
        return q

## 13. MaxPool2d [Advanced]
- input:   **`batch_size x n_input_channels x h x w`**
- output: **`batch_size x n_output_channels x h // kern_size x w // kern_size`**

You are to implement simplified version of pytorch `MaxPool2d` layer with stride = kernel_size. Please note, that it's not a common case that stride = kernel_size: in AlexNet and ResNet kernel_size for max-pooling was set to 3, while stride was set to 2. We introduce this restriction to make implementation simplier.

Practical notes:
- During forward pass what you need to do is just to reshape the input tensor to `[n, c, h / kern_size, kern_size, w / kern_size, kern_size]`, swap two axes and take maximums over the last two dimensions. Reshape + axes swap is sometimes called space-to-batch transform.
- During backward pass you need to place the gradients in positions of maximal values taken during the forward pass
- In real frameworks the indices of maximums are stored in memory during the forward pass. It is cheaper than to keep the layer input in memory and recompute the maximums.

In [38]:
class MaxPool2d(Module):
    def __init__(self, kernel_size):
        super(MaxPool2d, self).__init__()
        self.kernel_size = kernel_size
        self.gradInput = None
        self.max_indices = None

    def updateOutput(self, input):
        batch_size, channels, height, width = input.shape

        assert height % self.kernel_size == 0
        assert width % self.kernel_size == 0

        new_h = height // self.kernel_size
        new_w = width // self.kernel_size

        # Reshape входного тензора для векторизованной операции max pooling
        reshaped = input.reshape(
            batch_size, channels, new_h, self.kernel_size, new_w, self.kernel_size
        )

        self.output = reshaped.max(axis=(3, 5))

        reshaped_for_indices = reshaped.reshape(
            batch_size, channels, new_h, new_w, self.kernel_size * self.kernel_size
        )
        self.max_indices = reshaped_for_indices.argmax(axis=4)

        return self.output

    def updateGradInput(self, input, gradOutput):
        batch_size, channels, height, width = input.shape
        new_h = height // self.kernel_size
        new_w = width // self.kernel_size

        self.gradInput = np.zeros_like(input)

        # Векторизованная версия распространения градиентов
        batch_idx = np.arange(batch_size).repeat(channels * new_h * new_w)

        channel_idx = np.tile(np.arange(channels).repeat(new_h * new_w), batch_size)

        block_h_idx = np.tile(np.arange(new_h).repeat(new_w), batch_size * channels)

        block_w_idx = np.tile(np.arange(new_w), batch_size * channels * new_h)

        flat_indices = self.max_indices.ravel()

        max_h = flat_indices // self.kernel_size
        max_w = flat_indices % self.kernel_size

        orig_h = block_h_idx * self.kernel_size + max_h
        orig_w = block_w_idx * self.kernel_size + max_w

        self.gradInput[batch_idx, channel_idx, orig_h, orig_w] = gradOutput.ravel()

        return self.gradInput

    def __repr__(self):
        q = 'MaxPool2d, kern %d, stride %d' %(self.kernel_size, self.kernel_size)
        return q

In [37]:
def test_conv2d():
    # Создаем тестовые данные
    batch_size = 2
    in_channels = 3
    out_channels = 2
    height, width = 5, 5
    kernel_size = 3

    # Создаем случайное изображение
    input_data = np.random.randn(batch_size, in_channels, height, width)

    # Создаем слой Conv2d
    conv = Conv2d(in_channels, out_channels, kernel_size)

    # Forward pass
    output = conv.forward(input_data)
    print(f"Input shape: {input_data.shape}")
    print(f"Output shape: {output.shape}")

    # Проверка размерности выхода
    assert output.shape == (batch_size, out_channels, height, width), \
        f"Expected shape {(batch_size, out_channels, height, width)}, got {output.shape}"

    # Backward pass
    grad_output = np.ones_like(output)
    grad_input = conv.backward(input_data, grad_output)

    print(f"GradInput shape: {grad_input.shape}")
    assert grad_input.shape == input_data.shape, "GradInput shape mismatch"

    # Проверка градиентов параметров
    conv.accGradParameters(input_data, grad_output)
    print(f"GradW shape: {conv.gradW.shape}")
    print(f"Gradb shape: {conv.gradb.shape}")

    print("✓ Conv2d test passed!")

def test_maxpool2d():
    # Тестируем MaxPool2d
    batch_size = 2
    channels = 3
    height, width = 6, 6
    kernel_size = 2

    # Создаем случайное изображение
    input_data = np.random.randn(batch_size, channels, height, width)

    # Создаем слой MaxPool2d
    pool = MaxPool2d(kernel_size)

    # Forward pass
    output = pool.forward(input_data)
    print(f"Input shape: {input_data.shape}")
    print(f"Output shape: {output.shape}")

    expected_h = height // kernel_size
    expected_w = width // kernel_size
    assert output.shape == (batch_size, channels, expected_h, expected_w), \
        f"Expected shape {(batch_size, channels, expected_h, expected_w)}, got {output.shape}"

    # Backward pass
    grad_output = np.ones_like(output)
    grad_input = pool.backward(input_data, grad_output)

    print(f"GradInput shape: {grad_input.shape}")
    assert grad_input.shape == input_data.shape, "GradInput shape mismatch"

    print("✓ MaxPool2d test passed!")

# Запуск тестов
print("Testing Conv2d...")
test_conv2d()
print("\nTesting MaxPool2d...")
test_maxpool2d()

Testing Conv2d...
Input shape: (2, 3, 5, 5)
Output shape: (2, 2, 5, 5)
GradInput shape: (2, 3, 5, 5)
GradW shape: (2, 3, 3, 3)
Gradb shape: (2,)
✓ Conv2d test passed!

Testing MaxPool2d...
Input shape: (2, 3, 6, 6)
Output shape: (2, 3, 3, 3)
GradInput shape: (2, 3, 6, 6)
✓ MaxPool2d test passed!


### Flatten layer
Just reshapes inputs and gradients. It's usually used as proxy layer between Conv2d and Linear.

In [31]:
class Flatten(Module):
    def __init__(self):
         super(Flatten, self).__init__()

    def updateOutput(self, input):
        self.output = input.reshape(len(input), -1)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput.reshape(input.shape)
        return self.gradInput

    def __repr__(self):
        return "Flatten"

In [32]:
def test_flatten():
    print("Testing Flatten layer...")

    # Тест 1: 4D тензор (после сверточного слоя)
    batch_size = 4
    channels = 3
    height, width = 5, 5

    # Создаем входной тензор
    input_4d = np.random.randn(batch_size, channels, height, width)
    flatten = Flatten()

    # Forward pass
    output = flatten.forward(input_4d)
    expected_size = batch_size * channels * height * width
    print(f"4D Input shape: {input_4d.shape}")
    print(f"Output shape: {output.shape}")
    print(f"Expected flattened size: {expected_size}")

    # Проверяем размер
    assert output.shape == (batch_size, channels * height * width), \
        f"Expected shape {(batch_size, channels * height * width)}, got {output.shape}"

    # Проверяем, что данные сохраняются
    assert np.allclose(output[0, 0], input_4d[0, 0, 0, 0]), "Data mismatch!"

    # Backward pass
    grad_output = np.ones_like(output)
    grad_input = flatten.backward(input_4d, grad_output)

    # Проверяем форму градиента
    assert grad_input.shape == input_4d.shape, "GradInput shape mismatch"

    # Тест 2: 3D тензор
    input_3d = np.random.randn(batch_size, height, width)
    output_3d = flatten.forward(input_3d)
    print(f"\n3D Input shape: {input_3d.shape}")
    print(f"Output shape: {output_3d.shape}")

    grad_output_3d = np.ones_like(output_3d)
    grad_input_3d = flatten.backward(input_3d, grad_output_3d)
    assert grad_input_3d.shape == input_3d.shape, "3D GradInput shape mismatch"

    print("✓ Flatten test passed!")

# Запуск теста
test_flatten()

Testing Flatten layer...
4D Input shape: (4, 3, 5, 5)
Output shape: (4, 75)
Expected flattened size: 300

3D Input shape: (4, 5, 5)
Output shape: (4, 25)
✓ Flatten test passed!
